# API Interaction

## Objectives
1. Demonstrate how to interact with the IMDB-based API generated earlier.
2. Retrieve and process data about directors and movies.
3. Show how to handle nested JSON responses and convert them to pandas DataFrames.

## Key Techniques Demonstrated
- API requests using the `requests` library
- JSON data processing
- Handling nested data structures
- Pandas DataFrame manipulation (json_normalize, merging, type conversion)

## Contents
1. Import packages
2. Test the API connection
3. Data retrieval and manipulation

----
## 1. Import packages

In [5]:
import requests
import pandas as pd

----
## 2. Test API connection

In [6]:
# Define the API endpoint URL for the Recommendation system based on IMDB data
api_imdb = "https://henry-mlops-imdb.onrender.com/"

# Send a GET request to the IMDB API endpoint
response_api = requests.get(api_imdb)

# Print the raw text response from the API
print(response_api.text)

{"message":"API del proyecto: Henry PI_MLOPS, por mmvvff"}


----
## 3. Data retrieval and manipulation

### 3.1. Director Information Retrieval
First, we fetch data about a specific director (Woody Allen).


In [7]:
# Define the API URL with placeholder for director name
url_imdb_director = "https://henry-mlops-imdb.onrender.com/director_exitoso/{}"

# Define the director name
query_director = "Woody Allen"

# Format the API URL with the director name
api_imdb_director = url_imdb_director.format(query_director)

# Send the GET request using requests
response_api_director = requests.get(api_imdb_director)

data_director_query = pd.DataFrame(response_api_director.json())
data_director_query.head(3)

,director,retorno total,peliculas
0,Woody Allen,3.02,"{'nombre': 'Mighty Aphrodite', 'fecha_lanzamie..."
1,Woody Allen,3.02,"{'nombre': 'Bullets Over Broadway', 'fecha_lan..."
2,Woody Allen,3.02,"{'nombre': 'Manhattan Murder Mystery', 'fecha_..."


Second, we process the nested JSON response.
Third, we create a structured DataFrame with director and movie information

In [8]:
# Set the index of the DataFrame to the 'director' column
data_director_query_nested = data_director_query.set_index("director")

# Unnest the nested list using pandas.json_normalize and set the index to the original index
data_director_query_unnested = pd.json_normalize(
    data_director_query_nested["peliculas"]).set_index(data_director_query_nested.index)

# Merge the 'director' information DataFrame (excluding the 'peliculas' column and duplicates)
# with the unnested movie DataFrame on the 'director' index
data_director_expanded = pd.merge(
    data_director_query.drop(columns=["peliculas"]).drop_duplicates().set_index("director"),
    data_director_query_unnested,
    left_index=True, right_index=True
# Reset the index to create a new column for the index and convert data types
).reset_index().convert_dtypes()

data_director_expanded.info()
data_director_expanded.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   director           49 non-null     string 
 1   retorno total      49 non-null     Float64
 2   nombre             49 non-null     string 
 3   fecha_lanzamiento  49 non-null     string 
 4   costo (USD)        49 non-null     string 
 5   ganancia (USD)     49 non-null     string 
 6   retorno (ratio)    49 non-null     Float64
dtypes: Float64(2), string(5)
memory usage: 2.9 KB


,director,retorno total,nombre,fecha_lanzamiento,costo (USD),ganancia (USD),retorno (ratio)
0,Woody Allen,3.02,Mighty Aphrodite,1995-09-13,"15,000,000","6,700,000",0.45
1,Woody Allen,3.02,Bullets Over Broadway,1994-10-14,0,0,0.0
2,Woody Allen,3.02,Manhattan Murder Mystery,1993-08-18,"13,500,000","11,285,588",0.84
3,Woody Allen,3.02,Everyone Says I Love You,1996-12-06,"20,000,000",0,0.0
4,Woody Allen,3.02,Sleeper,1973-12-17,"2,000,000","18,344,729",9.17


### 3.2. Movie Recommendations
We retrieve movie recommendations based on a given movie title, and display the recommendations in a pandas DataFrame.

In [10]:
# Define the API URL with placeholder for director name
url_imdb_recommend = "https://henry-mlops-imdb.onrender.com/recomendacion_pelicula/{}"

# Define the director name
query_movie = "Top Gun"

# Format the API URL with the director name
api_imdb_movie = url_imdb_recommend.format(query_movie)

# Send the GET request using requests
response_api_movie = requests.get(api_imdb_movie)

data_movie_query = pd.DataFrame(response_api_movie.json())
data_movie_query.head(5)

,pelicula,recomendaciones
0,Top Gun,Always
1,Top Gun,Blue Chips
2,Top Gun,For the Moment
3,Top Gun,Area 88
4,Top Gun,Return with Honor
